# Exercise 1


_course: quantum cyrptography for beginners
<br>date: 23 august 2020
<br>author: burton rosenberg_

In this exercise we look at three quantum gates: X, H and CNOT.


## Installation

In order to bring this to your computer, you must install [Anaconda](https://docs.anaconda.com/anaconda/install/), which will bring you,

- Python 3 and a large number of scientific pages such has numpy,
- Jupyter, the scientific notebook which runs in your browser, and makes and executes pages like this,
- Conda, a package management system, including the ability to have virtual python environments.

Then you will need the [qiskit package](https://qiskit.org/documentation/install.html). Qiskit is installed with pip, rather than conda.

A great walk-through is [Episode 2](https://www.youtube.com/watch?v=M4EkW4VwhcI&vl=en-US) of Qiskit live, with Abraham Asfaw. Note that his walk-through does not create a qiskit conda environment, as described in the qiskit installation documentation. Either seems fine.


In [1]:

import qiskit
import time, math

from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from qiskit.providers.jobstatus import JOB_FINAL_STATES, JobStatus
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, execute, Aer

qiskit.__qiskit_version__

{'qiskit-terra': '0.15.1',
 'qiskit-aer': '0.6.1',
 'qiskit-ignis': '0.4.0',
 'qiskit-ibmq-provider': '0.8.0',
 'qiskit-aqua': '0.7.5',
 'qiskit': '0.20.0'}

### Loading your account and listing backends

You need to enroll in the IBM Quantum Experience and get a token to access the API. In the following code, you only need to run load_or_save_IBMQ_account once with the token. Then it will be saved in your home directory.

You can list the machines (backends) that you have access to.

In [2]:

args_g = []

# your api token from IBM, first time run.
# after that None is good

#api_token = 'abcdefghijklmnopqrstuvwxyz'
api_token = None 

def load_or_save_IBMQ_account(api_token=None):
    global args_g
    if api_token:
        # only needs to be done once
        # then is stored in e.g. ~/.qistkit/qiskitrc
        IBMQ.save_account(api_token)
    provider = IBMQ.load_account()
    return provider

def list_backends(provider):
    global args_g
    backends = provider.backends()
    print('backends available:')
    for be in backends:
        st = be.status()
        if st.operational:
            print(f'\t{be.name()}, pending jobs:{st.pending_jobs}')

            
def run_quantum_circuit_on_backend(quantum_circuit,provider,backend):
    backend = provider.get_backend(backend)
    qobj = assemble(transpile(quantum_circuit, backend=backend), backend=backend)
    job = backend.run(qobj)
    return job


def wait_for_job(backend, job, wait_interval=5):
    backend = provider.get_backend(backend)
    retrieved_job = backend.retrieve_job(job.job_id())
    start_time = time.time()
    job_status = job.status()
    while job_status not in JOB_FINAL_STATES:
        print(f'Status @ {time.time() - start_time:0.0f} s: {job_status.name},'
              f' est. queue position: {job.queue_position()}')
        time.sleep(wait_interval)
        job_status = job.status()


provider = load_or_save_IBMQ_account(api_token)
list_backends(provider)

/Users/ojo/anaconda3/lib/python3.6/site-packages/qiskit/providers/ibmq/ibmqfactory.py:192: UserWarning: Timestamps in IBMQ backend properties, jobs, and job results are all now in local time instead of UTC.
  warnings.warn('Timestamps in IBMQ backend properties, jobs, and job results '


backends available:
	ibmq_qasm_simulator, pending jobs:0
	ibmqx2, pending jobs:5
	ibmq_16_melbourne, pending jobs:19
	ibmq_vigo, pending jobs:86
	ibmq_ourense, pending jobs:214
	ibmq_valencia, pending jobs:26
	ibmq_london, pending jobs:96
	ibmq_burlington, pending jobs:36
	ibmq_essex, pending jobs:12
	ibmq_armonk, pending jobs:0
	ibmq_santiago, pending jobs:21


In [3]:
# choose your backend

backend = 'ibmq_qasm_simulator'
#backend = 'ibmq_armonk'
#backend = 'ibmq_vigo'
#backend = 'ibmq_london'

# and so forth ... chose from the results given by provider.backends()

## Quantum Circuits, Quantum Gates

### H: The Hadamard Gate

Quantum computation in the model we are considering, is the construction of quantum circuits out of quantum gates. The inputs and output to the gates a quantum bits, called qubits. 

It is helpful to be familiar with classical bits and digital logic gates. Those gates implement the logical functions of and, or and not. These are the operations in a boolean algebra where the numbers are True and False, and the operations are and, or a not. All classical computers are built from those basic gates combined over a billion times in billions of ways.

Quantum computation takes this as a model idea, but uses quantum bits, or qubits, and quantum gates, that are created by developing a Hamiltonian (not the Broadway Play) around the qubit letting it evolve by Schödinger's equation.

The first gate we shall learn is the H gate, short for Hadamard Gate, named after Jaques Hadamard (1865&ndash;1963) a French mathematcian that briefly taught at Columbia University in New York City. 

Th gate takes one qubit input and gives a one qubit output that is a superposition of two states &mdash; it is both true and false at the same time. It is a good starting point for thinking about and understanding the quantum phenomenon of superposition.

Classically, a bit is either 0 or 1. What explains quantum behavior best is that a vector of possibilities can be established, and unless measured, the quantum bit can evolve simultaneously along each possibility, and the outcomes of the computation are not only values, but probabilities of obtaining those values, when the qubit is measured.

Measurement will provide information to a world that can only understand classical information, and hence once measured, and the qubit decides in retrospect which quantum superposed path to reveal to the world, all other pathes become an historical untruth. There is no way of knowing anything about those other paths. The veil between the quantum world and the classical world is a Rule of Nature, and we cannot break from this rule.

Formally, the state of a qubit is a vector in a complex space of dimension two. One dimension is called $|0\rangle$, and the other $|1\rangle$. The Bloch Sphere depicts one as up and the other down, and when realized as, say the spin on an electron (a Fermion), one is "up" and the other "down".

Despite the suggestion that up and down are not orthgonal (at right angles), but opposite, up and down are, as vectors in the complex vector space of dimention two, $\cal{C}^2$, orthogonal. So the plus state, given by the Hadamard gate acting on a $|0\rangle$, gives the minus state when acting on a $|1\rangle$.


In [4]:

def make_a_plus_state():
    # make the circuit
    q = QuantumRegister(1)
    c = ClassicalRegister(1)
    quantum_circuit = QuantumCircuit(q, c)
    quantum_circuit.h(q)
    quantum_circuit.measure(q, c)
    print('\n-------- CIRCUIT ---------')
    print(quantum_circuit.draw(output='text'))
    print('-------------------------\n')
    return quantum_circuit        
# our first circuit, comprising of a single H gate and a measurement

quantum_circuit = make_a_plus_state()

job = run_quantum_circuit_on_backend(quantum_circuit,provider,backend)
print(f'results: waiting for results from backend {backend} ...')
wait_for_job(backend, job)
result = job.result()
print(f'results: {result.get_counts()}')

   


-------- CIRCUIT ---------
      ┌───┐┌─┐
q0_0: ┤ H ├┤M├
      └───┘└╥┘
c0: 1/══════╩═
            0 
-------------------------

results: waiting for results from backend ibmq_qasm_simulator ...
Status @ 0 s: VALIDATING, est. queue position: None
results: {'0': 517, '1': 507}


### X: The Pauli X Gate

The inputs are not generally termed true and false, in the logic of qubits, but |0> and |1>. The bar and angle surrounding some identifier is a notation invented by PAM Dirac, and is called a Ket. The previous example took the zero state, |0>, passed it through an Hadamard gate, and got a superpostion of |0> and |1> called the plus state and noted as |+>. In math,

$$
 |+\rangle = H\, |0\rangle
$$

The Pauli X Operator performs a logical not, so that |0> is transformed in passing through X to |1>, and |1> to |0>.

So we would also expect that casading two X's give the identity,

$$
 |0\rangle = X\,X\, |0\rangle, \;\; |1\rangle = X\,X\, |1\rangle
$$

References: [Qiskit X Gate](https://qiskit.org/documentation/stubs/qiskit.circuit.library.XGate.html#qiskit.circuit.library.XGate)

In [5]:

def make_a_minus_state():
    # make the circuit
    q = QuantumRegister(1)
    c = ClassicalRegister(1)
    quantum_circuit = QuantumCircuit(q, c)
    quantum_circuit.x(q)
    quantum_circuit.h(q)
    quantum_circuit.measure(q, c)
    print('\n-------- CIRCUIT ---------')
    print(quantum_circuit.draw(output='text'))
    print('-------------------------\n')
    return quantum_circuit        
# our first circuit, comprising of a single H gate and a measurement

quantum_circuit = make_a_minus_state()

job = run_quantum_circuit_on_backend(quantum_circuit,provider,backend)
print(f'results: waiting for results from backend {backend} ...')
wait_for_job(backend, job)
result = job.result()
print(f'results: {result.get_counts()}')


def make_a_minus_state_measure_as_minus():
    # make the circuit
    q = QuantumRegister(1)
    c = ClassicalRegister(1)
    quantum_circuit = QuantumCircuit(q, c)
    quantum_circuit.x(q)
    quantum_circuit.h(q)
    quantum_circuit.h(q)
    quantum_circuit.measure(q, c)
    print('\n-------- CIRCUIT ---------')
    print(quantum_circuit.draw(output='text'))
    print('-------------------------\n')
    return quantum_circuit        
# our first circuit, comprising of a single H gate and a measurement

quantum_circuit = make_a_minus_state_measure_as_minus()

job = run_quantum_circuit_on_backend(quantum_circuit,provider,backend)
print(f'results: waiting for results from backend {backend} ...')
wait_for_job(backend, job)
result = job.result()
print(f'results: {result.get_counts()}')



-------- CIRCUIT ---------
      ┌───┐┌───┐┌─┐
q3_0: ┤ X ├┤ H ├┤M├
      └───┘└───┘└╥┘
c1: 1/═══════════╩═
                 0 
-------------------------

results: waiting for results from backend ibmq_qasm_simulator ...
Status @ 0 s: VALIDATING, est. queue position: None
results: {'0': 523, '1': 501}

-------- CIRCUIT ---------
      ┌───┐┌───┐┌───┐┌─┐
q7_0: ┤ X ├┤ H ├┤ H ├┤M├
      └───┘└───┘└───┘└╥┘
c2: 1/════════════════╩═
                      0 
-------------------------

results: waiting for results from backend ibmq_qasm_simulator ...
Status @ 0 s: VALIDATING, est. queue position: None
results: {'1': 1024}


### Exercise A:

Create a quantum circuit that gives the superposition of all 4 possible states of a qubit pair: 00, 01, 10, and 11. 

Hint: Just two H gates will do it.

Notation: One can associated a binary number with a configuration of inputs, taking care not to discard leading zeros The case of 2 qubits as four binary numbers,

$$
|00\rangle, \; |01\rangle, \;  |10\rangle, \; |11\rangle 
$$

or they can be written in decimal, again remembering how many binary leading zeros to retain:

$$
|0\rangle, \; |1\rangle, \;  |2\rangle, \; |3\rangle 
$$

As a convention, the least significant bit is associated with the top qubit in the diagrams that arranges the input qubits vertically at the left edge of the diagram.

_sample output_

<pre><font color=red>
results: waiting for results from backend ibmq_qasm_simulator ...
Status @ 0 s: VALIDATING, est. queue position: None
results: {'00': 266, '01': 257, '10': 258, '11': 243}
</font></pre>

### Cnot: The Controlled Not Gate

The Cnot gate takes two qubits as input and produces two qubits as output. One input qubit is the control, and it operates by negating the other input, if one; else both input pass unchanged,

$$
|ab\rangle = |a(a+b)\rangle
$$

where plus is defined as exclusive or, or equivalently mod 2 addition. 

References: [CX Gate](https://qiskit.org/documentation/stubs/qiskit.circuit.library.CXGate.html), [CNOT](https://qiskit.org/textbook/ch-gates/multiple-qubits-entangled-states.html#cnot)


In [8]:

def make_qubit_pair():
    # make the circuit
    q = QuantumRegister(2)
    c = ClassicalRegister(2)
    return (QuantumCircuit(q, c),q,c)


def add_measurement_qubit_pair(qubit_pair):
    qubit_pair[0].measure(qubit_pair[1],qubit_pair[2])
    print('\n-------- CIRCUIT ---------')
    print(qubit_pair[0].draw(output='text'))
    print('-------------------------\n')
    return qubit_pair


def add_cnot_qubit_pair(qubit_pair):
    qubit_pair[0].cx(0,1)
    return qubit_pair

def add_x_qubit_pair(qubit_pair):
    qubit_pair[0].x(0)
    return qubit_pair


# with a |0> input

qc = make_qubit_pair()
qc = add_cnot_qubit_pair(qc)
qc = add_measurement_qubit_pair(qc)

job = run_quantum_circuit_on_backend(qc[0],provider,backend)
print(f'results: waiting for results from backend {backend} ...')
wait_for_job(backend, job)
result = job.result()
print(f'results: {result.get_counts()}')

# with a |1> input

qc = make_qubit_pair()
qc = add_x_qubit_pair(qc)
qc = add_cnot_qubit_pair(qc)
qc = add_measurement_qubit_pair(qc)

job = run_quantum_circuit_on_backend(qc[0],provider,backend)
print(f'results: waiting for results from backend {backend} ...')
wait_for_job(backend, job)
result = job.result()
print(f'results: {result.get_counts()}')




-------- CIRCUIT ---------
            ┌─┐   
q12_0: ──■──┤M├───
       ┌─┴─┐└╥┘┌─┐
q12_1: ┤ X ├─╫─┤M├
       └───┘ ║ └╥┘
 c4: 2/══════╩══╩═
             0  1 
-------------------------

results: waiting for results from backend ibmq_qasm_simulator ...
Status @ 0 s: VALIDATING, est. queue position: None
results: {'00': 1024}

-------- CIRCUIT ---------
       ┌───┐     ┌─┐   
q15_0: ┤ X ├──■──┤M├───
       └───┘┌─┴─┐└╥┘┌─┐
q15_1: ─────┤ X ├─╫─┤M├
            └───┘ ║ └╥┘
 c5: 2/═══════════╩══╩═
                  0  1 
-------------------------

results: waiting for results from backend ibmq_qasm_simulator ...
Status @ 0 s: VALIDATING, est. queue position: None
results: {'11': 1024}



### Exercise B

Show the action of the cnot on all four inputs, $|0\rangle$ through $|3\rangle$, using the subroutine you wrote for Exercise A.


_sample output_

<pre><font color=red>
results: waiting for results from backend ibmq_qasm_simulator ...
Status @ 1 s: RUNNING, est. queue position: None
results: {'00': 226, '01': 254, '10': 274, '11': 270}
</font></pre>

## Combining Quantum Gates

### An entangled qubit pair

Superposition is the quantum condition of qubits being simulatneously in several states. It is possible to create superpositions that link qubits together, that is, such that measuring one qubit, thus determining the state of that qubit, will determine the state of a different qubit, when prior it was in superposition.

Such as state is the EPR pair, which is a two qubit state, a superposition of $|00\rangle$ and $|11\rangle$.

Consider the first qubit: it is simultaneously $|0\rangle$ and $|1\rangle$ until measured. The same is true of the second qubit. However if either qubit is measured and the result is $|0\rangle$, then the state must be $|00\rangle$ and ttaking the measurement of other qubit will result in $|0\rangle$.. If either qubit is measured and the result is $|1\rangle$, then the state must be $|11\rangle$ and taking the measurement of other qubit will result in $|1\rangle$.


To create such a pair, attach an H gate to the control of a cnot.


In [ ]:
qp = make_qubit_pair()
qp[0].h(0)
qp = add_cnot_qubit_pair(qp)
qp = add_measurement_qubit_pair(qp)

job = run_quantum_circuit_on_backend(qp[0],provider,backend)
print(f'results: waiting for results from backend {backend} ...')
wait_for_job(backend, job)
result = job.result()
print(f'results: {result.get_counts()}')



### Exercise C

Give a circuit that creates the entangled pair $|01\rangle$ with $|10\rangle$.


_sample output_

<pre><font color=red>
results: waiting for results from backend ibmq_qasm_simulator ...
Status @ 1 s: RUNNING, est. queue position: None
results: {'01': 525, '10': 499}
</font></pre>

### Exercise D

Create a three qubit circuit that puts the three qubits in Greenberger–Horne–Zeilinger state (GHZ state).

For 3 qubits, this is the superposition of $|000\rangle$ with $|111\rangle$.

_sample output_

<pre><font color=red>
results: waiting for results from backend ibmq_qasm_simulator ...
Status @ 0 s: VALIDATING, est. queue position: None
results: {'000': 512, '111': 512}
</font></pre>

### Exercise E

Give a circuit that swaps the values of two qubits. That is, if the input is $|ab\rangle$, the output is $|ba\rangle$, where a and b are 0 or 1. Hint: use three cnot gates.


_sample output_

<pre><font color=red>
input: 00
results: waiting for results from backend ibmq_qasm_simulator ...
Status @ 0 s: QUEUED, est. queue position: None
results: {'00': 1024}

input: 01
results: waiting for results from backend ibmq_qasm_simulator ...
Status @ 0 s: VALIDATING, est. queue position: None
results: {'10': 1024}

input: 10
results: waiting for results from backend ibmq_qasm_simulator ...
Status @ 0 s: QUEUED, est. queue position: None
results: {'01': 1024}

input: 11
results: waiting for results from backend ibmq_qasm_simulator ...
Status @ 0 s: VALIDATING, est. queue position: None
results: {'11': 1024}
</font></pre>


### Exercise F

Give a three qubit circuit that has output the exclusive or of all three input qubits. The result replaces the value of the middle qubit. 

Run it on a superposed input mixing $|000\rangle$, $|001\rangle$, $|100\rangle$, $|101\rangle$. Explain why the output suggests superposed outputs of four computations.


_sample output_

<pre><font color=red>
results: waiting for results from backend ibmq_qasm_simulator ...
Status @ 0 s: VALIDATING, est. queue position: None
results: {'000': 266, '011': 236, '101': 252, '110': 270}
</font></pre>
